In [1]:
sst_dataset = {}
for split in ["train", "dev", "test"]:
    URL = f"https://raw.githubusercontent.com/successar/instance_attributions_NLP/master/Datasets/SST/data/{split}.jsonl"
    import urllib.request, json 
    with urllib.request.urlopen(URL) as url:
        data = url.read().decode()
        data = [json.loads(line) for line in data.strip().split("\n")]
        sst_dataset[split] = data

In [2]:
import numpy as np
from tqdm import tqdm

In [3]:
import spacy 
nlp = spacy.load('en_core_web_md')

# We use spacy to get feature vectors for our input text.

In [4]:
import numpy as np
from tqdm import tqdm
X, y = {}, {}
for split in ["train", "dev"]:
    X[split] = np.array([nlp(example["document"]).vector for example in tqdm(sst_dataset[split])])
    y[split] = np.array([example["label"] for example in sst_dataset[split]])

100%|████████████████████████████████████████| 872/872 [00:06<00:00, 132.82it/s]


In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.7924311926605505

In [6]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
g = np.sum(gradient_train, axis=0) / 6920
gradient_train.shape, gradient_dev.shape

((6920, 301), (872, 301))

In [7]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 1 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [8]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(872, 6920)

In [9]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    print("prediction", prediction)

    return X_k, y_k, prediction

In [10]:
def show_influential(dev_idx, k, scores=delta_pred):
    if pred[dev_idx] > 0.5:
        top_k_index = scores[dev_idx].argsort()[-k:]
    else:
        top_k_index = scores[dev_idx].argsort()[:k]
    
    item = sst_dataset["dev"][dev_idx]
    print(f"label: {item['label']} doc: {item['document']}")

    print("Top Example: ========")
    for idx in top_k_index:
        item = sst_dataset["train"][idx]
        #print("Score: ", scores[dev_idx][idx])
        print(f"label: {item['label']} doc: {item['document']}")


In [11]:
show_influential(1, 10, scores=delta_pred)

label: 1 doc: and if you re not nearly moved to tears by a couple of scenes you ve got ice water in your veins
Top Example: ========
label: 0 doc: a few hours after you ve seen it you forget you ve been to the movies
label: 0 doc: a chiller resolutely without chills
label: 1 doc: exhilarating but blatantly biased
label: 0 doc: roman polanski directs the pianist like a surgeon mends a broken heart very meticulously but without any passion
label: 1 doc: a bodice ripper for intellectuals
label: 0 doc: a beautifully shot but dull and ankle deep epic
label: 0 doc: you ll cry for your money back
label: 1 doc: even during the climactic hourlong cricket match boredom never takes hold
label: 0 doc: although visually striking and slickly staged it s also cold grey antiseptic and emotionally desiccated
label: 0 doc: run for your lives


In [12]:
app_k = np.load("appro_ks_IP.npy", allow_pickle=True)
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if app_k[i] == None:
        app_k_nonon.append(6920)
    else:
        app_k_nonon.append(app_k[i])

None
53
None
None
None
None
None
49
213
172
718
126
435
580
20
18
142
54
None
None
10
None
141
None
None
None
137
30
11
1
None
11
2
6
None
2718
25
60
26
123
None
43
216
None
642
None
767
214
None
None
105
2621
61
31
None
12
None
8
2
18
618
410
465
None
558
None
333
53
None
137
73
425
None
None
77
None
82
289
266
5
None
None
None
775
22
17
None
1786
1144
425
497
45
83
13
114
2
None
1352
236
None
66
None
254
None
97
None
1
353
170
29
248
1
15
36
84
679
284
14
34
1
13
396
13
384
33
50
165
None
3
86
140
None
None
None
19
155
74
12
11
48
87
51
None
None
57
2265
None
1368
751
669
31
471
130
74
118
None
None
1273
64
None
24
17
1
60
2
None
None
19
11
11
None
None
None
202
6
44
433
1734
253
None
4
5
1747
None
241
58
100
1750
None
375
59
6
65
591
660
168
None
None
339
332
2058
78
24
559
107
None
589
852
None
1126
4
None
66
53
3
3
534
14
11
19
84
None
101
None
16
504
243
35
35
1276
104
None
170
73
3
28
1185
None
149
78
1
None
157
166
95
219
17
304
32
2263
None
445
None
32
None
None
21
22
14
722
N

In [13]:
sort_dev_k = np.array(app_k_nonon).argsort()
sort_dev_k

array([662, 661, 630, 510, 106, 750, 492, 670,  29, 240, 636, 111, 162,
       792, 421, 401, 119, 790, 859,  95, 539, 164, 782, 445, 547,  58,
        32, 801, 819, 128, 728, 495, 713, 215, 572, 214, 234, 653, 680,
       210, 762, 631, 648, 867, 267, 180, 649, 426, 621, 285, 763,  79,
       659, 827, 321, 535, 181, 837, 403, 858, 448, 174, 281, 772, 667,
        33, 803, 324, 327, 191, 563, 841, 469, 459,  57, 734, 530, 727,
       556, 476, 592, 384, 868, 840, 616, 768, 628, 610, 665, 395,  20,
       294, 218, 168, 169,  28,  31, 138, 828, 614,  55, 344, 615, 806,
       137, 730, 548, 122, 587, 658, 120,  93, 399, 857, 217, 117, 258,
       783, 516, 112, 865, 692, 799, 224, 542, 161, 629, 246, 618, 798,
       288, 780,  85,  59,  15, 345, 810, 435, 134, 570, 672, 219, 167,
       453, 261, 434,  14, 705, 463, 650, 681, 761, 256, 646, 807, 708,
       506, 480, 520, 449, 257, 303, 660,  84, 746, 489, 374, 514, 590,
       202, 802, 160,  36, 825, 264, 425, 382,  38, 602, 457, 61

In [15]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [16]:
new_predictions = np.load("new_predictions.npy", allow_pickle=True)

In [17]:
new_predictions[1], pred[1].item()

(0.5655931235222307, 0.3009748953355743)

In [18]:
data = []
for test_movie_id in sort_dev_k:
    k = app_k[test_movie_id]
    item = sst_dataset["dev"][test_movie_id]
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(item['label']) + "  doc: " + str(item['document']))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == item['label'])
    # k
    test_point.append(str(k))
    
    if k == None:
        data.append(test_point)
        continue
    test_point.append(np.round(new_predictions[test_movie_id]) != np.round(pred[test_movie_id].item()))
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        item = sst_dataset["train"][idx]
        test_point.append("id: " + str(idx) + "  label: " + str(item['label']) + "  doc: " + str(item['document']))
    data.append(test_point)

In [19]:
import csv

with open('CSV_SST_LR_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [ ]:
ls

In [44]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_SST/IF_lr_fix'